In [2]:
import numpy as np

In [3]:
a = np.array([-0.535, -0.267, 0.802]).T
b = np.array([0.707, -0.707, 0]).T
print(a@a, a@b)


1.000718 -0.189476


После стандартизации мы прогоняем регрессию стандартизованного  на стандартизованные регрессоры без константы (без w0)

In [4]:
import pandas as pd # для работы с DataFrame 

# загружаем датасет
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
boston_data = pd.read_csv('housing.csv', header=None, delimiter=r"\s+", names=column_names)
boston_data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [5]:
boston_data[['CHAS', 'LSTAT', 'CRIM', 'RM']].describe()

,CHAS,LSTAT,CRIM,RM
count,506.000000,506.000000,506.000000,506.000000
mean,0.069170,12.653063,3.613524,6.284634
std,0.253994,7.141062,8.601545,0.702617
min,0.000000,1.730000,0.006320,3.561000
25%,0.000000,6.950000,0.082045,5.885500
50%,0.000000,11.360000,0.256510,6.208500
75%,0.000000,16.955000,3.677083,6.623500
max,1.000000,37.970000,88.976200,8.780000


In [6]:
# составляем матрицу наблюдений и вектор целевой переменной
A = np.column_stack((np.ones(506), boston_data[['CHAS', 'LSTAT', 'CRIM','RM']]))
y = boston_data[['PRICE']]
# вычисляем OLS-оценку для коэффициентов без стандартизации
w_hat=np.linalg.inv(A.T@A)@A.T@y
print(w_hat.values)

[[-1.92052548]
 [ 3.9975594 ]
 [-0.58240212]
 [-0.09739445]
 [ 5.07554248]]


In [7]:
# составляем матрицу наблюдений без доп столбца из единиц
A = boston_data[['CHAS', 'LSTAT', 'CRIM', 'RM']]
y = boston_data[['PRICE']]
# стандартизируем векторы в столбцах матрицы А
A_cent = A - A.mean()
# указываем axis=0, тк по умолчанию норма считается для всей матрицы, а не для отдельного столбца
A_st = A_cent/np.linalg.norm(A_cent, axis=0)
A_st.describe().round(2)

,CHAS,LSTAT,CRIM,RM
count,506.00,506.00,506.00,506.00
mean,-0.00,-0.00,-0.00,-0.00
std,0.04,0.04,0.04,0.04
min,-0.01,-0.07,-0.02,-0.17
25%,-0.01,-0.04,-0.02,-0.03
50%,-0.01,-0.01,-0.02,-0.00
75%,-0.01,0.03,0.00,0.02
max,0.16,0.16,0.44,0.16


In [8]:
# длины всех векторов равны единице
np.linalg.norm(A_st, axis=0)

array([1., 1., 1., 1.])

Для получения стандартизированных коэффициентов нам также понадобится стандартизация целевой переменной y по тому же принципу:

In [9]:
y_cent = y - y.mean()
y_st = y_cent/np.linalg.norm(y_cent)

In [10]:
#
w_hat_st = np.linalg.inv(A_st.T@A_st)@A_st.T@y_st
w_hat_st.values

array([[ 0.11039956],
       [-0.45220423],
       [-0.09108766],
       [ 0.38774848]])

Чтобы проинтерпретировать оценки коэффициентов линейной регрессии (понять, каков будет прирост целевой переменной при изменении фактора на условную единицу), нам достаточно построить линейную регрессию в обычном виде без стандартизации и получить обычный вектор w_hat.

Однако, чтобы корректно говорить о том, какой фактор оказывает на прогноз большее влияние, необходимо рассматривать стандартизированную оценку вектора коэффициентов w_hat_st.

In [11]:
# матрица Грама для стандартизированных факторов
A_st.T@A_st
# и по факту это матрица выборочных корреляций наших исходных факторов

,CHAS,LSTAT,CRIM,RM
CHAS,1.000000,-0.053929,-0.055892,0.091251
LSTAT,-0.053929,1.000000,0.455621,-0.613808
CRIM,-0.055892,0.455621,1.000000,-0.219247
RM,0.091251,-0.613808,-0.219247,1.000000


Матрицу корреляций можно получить только в том случае, если производить стандартизацию признаков как векторы (делить на длину центрированного вектора x_st). Другие способы стандартизации/нормализации признаков не превращают матрицу Грама в матрицу корреляций.

In [12]:
# стандартизуем вектор
x = np.array([12, 8]).T
x_cent = x - x.mean()
x_st = x_cent/np.linalg.norm(x_cent)
x_st

array([ 0.70710678, -0.70710678])

In [13]:
x_1 = np.array([1, 2, 6])
x_2 = np.array([3000, 1000, 2000])
# считаем матрицу кореляций
np.corrcoef(x_1, x_2)

array([[ 1.        , -0.18898224],
       [-0.18898224,  1.        ]])

На практике корреляция с точки зрения линейной алгебры означает следующее:

Если корреляция cij =1, значит векторы xi и xj пропорциональны и сонаправлены.

Если корреляция cij =-1, значит векторы xi и xj пропорциональны и противонаправлены.

Если корреляция cij =0, значит векторы xi и xj ортогональны друг другу и, таким образом, являются линейно независимыми.

Корреляция — это мера линейной зависимости между признаками.

Чем больше по модулю корреляция между каким-нибудь фактором и целевым признаком, тем лучше

Чем больше по модулю корреляция между факторами, тем хуже 

Чем больше линейно зависимых факторов, тем меньше ранг.

In [14]:
v = np.array([5, 1, 2]).T
n = np.array([4, 2, 8]).T
np.corrcoef(v, n)
# коэффициент корреляции - на побочной диагонали

array([[1.        , 0.05241424],
       [0.05241424, 1.        ]])

In [24]:
x1 = np.array([5.1, 1.8, 2.1, 10.3, 12.1, 12.6]).T
x2 = np.array([10.2, 3.7, 4.1, 20.5, 24.2, 24.1]).T
x3 = np.array([2.5, 0.9, 1.1, 5.1, 6.1, 6.3]).T
a = np.corrcoef(x1, x2)
print(a, np.linalg.det(a))
b = np.corrcoef(x1, x3)
print(b, np.linalg.det(b))
c = np.corrcoef(x2, x3)
print(c, np.linalg.det(c))

[[1.         0.99925473]
 [0.99925473 1.        ]] 0.0014899747102504774
[[1.         0.99983661]
 [0.99983661 1.        ]] 0.0003267605292820508
[[1.         0.99906626]
 [0.99906626 1.        ]] 0.0018666044078268712


In [27]:
C = np.array([[1., 0.99925473, 0.99983661], [0.99925473, 1., 0.99906626], [0.99983661, 0.99906626, 1.]])
np.linalg.det(C)

np.float64(4.862222523854154e-07)